## Using multimodal 2d CNN and independent augmentations for each of the MRI types

This notebook stacks number of slices per each modality and performs independent data augmentations with the use of Monai framework for each MRI type. The used architecture is 2d Densenet121 initialized with the standard pre-trained weights.

Acknowledgements:

- https://www.kaggle.com/rluethy/efficientnet3d-with-one-mri-type
- https://www.kaggle.com/davidbroberts/determining-dicom-image-order
- https://www.kaggle.com/ihelon/brain-tumor-eda-with-animations-and-modeling
- https://www.kaggle.com/furcifer/torch-efficientnet3d-for-mri-no-train
- https://github.com/shijianjian/EfficientNet-PyTorch-3D
- https://www.kaggle.com/mikecho/rsna-miccai-monai-ensemble

This notebook is based on the framework Monai available here:
https://www.kaggle.com/mikecho/monai-v060-deep-learning-in-healthcare-imaging

Notebook to preprocess tensors:
https://www.kaggle.com/mikecho/rsna-miccai-preprocessing-160x160x160/


In [ ]:
import os
import sys 
import json
import glob
import random
import re
import collections
import time
import math
import logging

import numpy as np
import pandas as pd
import pydicom
import cv2
import matplotlib.pyplot as plt
import seaborn as sns

import torch
from torch import nn
from torch.utils import data as torch_data
from sklearn import model_selection as sk_model_selection
from torch.nn import functional as torch_functional
import torch.nn.functional as F
from torch.cuda.amp import GradScaler, autocast

from skimage.color import rgb2gray
from skimage import data
from skimage.filters import gaussian

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score

In [ ]:
!mkdir -p /root/.cache/torch/hub/checkpoints/
!cp ../input/densenet121pretrained/densenet121-a639ec97.pth /root/.cache/torch/hub/checkpoints/

In [ ]:
pip install --upgrade ../input/mclahenumpy/mclahe-numpy

In [ ]:
from mclahe import mclahe

In [ ]:
data_directory = '/kaggle/input/rsna-miccai-brain-tumor-radiogenomic-classification'
input_monaipath = "/kaggle/input/monai-v060-deep-learning-in-healthcare-imaging/MONAI-0.7.0"

parameter NUM_SLIZES indicates how many slices should be used for each modality

In [ ]:
mri_types = ['FLAIR', 'T1w', 'T1wCE', 'T2w']
BATCH_SIZE = 32
N_EPOCHS = 20
SEED = 42
BETA = 2.
LEARNING_RATE = 0.0002
LR_DECAY = 0.97
NUM_SLICES = 20
CHANNELS = NUM_SLICES * len(mri_types)

sys.path.append(input_monaipath)

from monai.networks.nets.densenet import DenseNet121

from monai.data import TestTimeAugmentation

from monai.transforms import (
    Compose,
    GibbsNoise,
    RandAffine,
    RandGibbsNoise,
    RandKSpaceSpikeNoise,
    RandRotate,
    apply_transform,
    RandAffined,
    ConcatItemsd,
    RandGibbsNoised,
    RandGaussianSmoothd
)

## Functions to load images

The parameter "randomized" is used for additional data augmentation to provide maximum variability between epochs.

Furthermore, with a probability 0.15 one of the modalities is removed (done twice).

In [ ]:
def load_2d_tensor(study_id, randomized=False):
    ds_path = f'../input/rsnamiccai-tensors-160x160x160-{1 if int(study_id) < 425 else 2}/kaggle/tmp/dataset/train'
    tensors_dict = {}
    
    mri_types_randomized = mri_types.copy()
    
    if randomized:
        if np.random.uniform() < 0.15:
            del mri_types_randomized[np.random.randint(0, len(mri_types_randomized))]

        if np.random.uniform() < 0.15:
            del mri_types_randomized[np.random.randint(0, len(mri_types_randomized))]
    
    for mri_type in mri_types:
        tensor_3d = np.load(f'{ds_path}/{study_id}/{mri_type}.npy')
        if mri_type not in mri_types_randomized:
            tensors_dict[mri_type] = np.zeros((NUM_SLICES, tensor_3d.shape[1], tensor_3d.shape[2]))
        else:
            tensors_list = []
            step = tensor_3d.shape[-1] / NUM_SLICES
            
            first_index = 0
            if randomized:
                first_index = np.random.randint(int(math.ceil(step)))

            for i in np.arange(first_index, tensor_3d.shape[-1], step):
                tensors_list.append(tensor_3d[0, :, :, int(i)])

            if len(tensors_list) > NUM_SLICES:
                tensors_list = tensors_list[:NUM_SLICES]
            elif len(tensors_list) < NUM_SLICES:
                for i in range(NUM_SLICES - len(tensors_list)):
                    tensors_list.append(tensors_list[-1])

            tensors_dict[mri_type] = np.stack(tensors_list, axis=0)
        
    return tensors_dict

In [ ]:
def set_seed(seed):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True

set_seed(SEED)

## train / test splits

In [ ]:
samples_to_exclude = [109, 123, 709]

train_df = pd.read_csv(f"{data_directory}/train_labels.csv")
print("original shape", train_df.shape)
train_df = train_df[~train_df.BraTS21ID.isin(samples_to_exclude)]
print("new shape", train_df.shape)
display(train_df)

df_train, df_valid = sk_model_selection.train_test_split(
    train_df, 
    test_size=0.2, 
    random_state=SEED, 
    stratify=train_df["MGMT_value"],
)


In [ ]:
df_train.tail()

## Model and training classes

In [ ]:
class Dataset(torch_data.Dataset):
    def __init__(self, paths, targets=None, transforms=None, split="train"):
        self.paths = paths
        self.targets = targets
        self.split = split
        self.transforms = transforms
        
        if transforms is not None:
            self.transforms.set_random_state(seed=SEED)
          
    def __len__(self):
        return len(self.paths)  #BATCH_SIZE * 2 # len(self.paths)
    
    def __getitem__(self, index):
        scan_id = self.paths[index]
        data = load_2d_tensor(str(scan_id).zfill(5), randomized=(self.split=='train'))
        
        if self.transforms is not None:
            data = apply_transform(self.transforms, data, map_items=False)
            data = data["img"]
            
        if not isinstance(data, torch.Tensor):
            data = torch.tensor(data, dtype=torch.float)
            
        if self.targets is None:
            return {"X": data, "id": scan_id}
        else:
            return {"X": data, "y": torch.tensor(self.targets[index], dtype=torch.float)}


In [ ]:
def build_model():
    model = DenseNet121(spatial_dims=2, in_channels=CHANNELS, out_channels=1, pretrained=True)
    return model    

In [ ]:
class Trainer:
    def __init__(
        self, 
        model, 
        device, 
        optimizer, 
        criterion
    ):
        self.model = model
        self.device = device
        self.optimizer = optimizer
        self.lr_scheduler = torch.optim.lr_scheduler.ExponentialLR(self.optimizer, gamma=LR_DECAY)
        self.criterion = criterion
        self.scaler = GradScaler()

        self.best_valid_score = .0
        self.n_patience = 0
        self.lastmodel = None
        
        self.val_losses = []
        self.train_losses = []
        self.val_auc = []
        self.val_auc_tta = []
        
    def fit(self, epochs, train_loader, valid_loader, save_path, patience):      
        for n_epoch in range(1, epochs + 1):
            self.info_message("EPOCH: {}", n_epoch)
            
            train_loss, train_time = self.train_epoch(train_loader)
            valid_loss, valid_auc, valid_time = self.valid_epoch(valid_loader)
            
            self.train_losses.append(train_loss)
            self.val_losses.append(valid_loss)
            self.val_auc.append(valid_auc)
            
            self.info_message(
                "[Epoch Train: {}] loss: {:.4f}, time: {:.2f} s            ",
                n_epoch, train_loss, train_time
            )
            
            self.info_message(
                "[Epoch Valid: {}] loss: {:.4f}, auc: {:.4f}, time: {:.2f} s",
                n_epoch, valid_loss, valid_auc, valid_time
            )

            if self.best_valid_score < valid_auc: 
                self.save_model(n_epoch, save_path, valid_loss, valid_auc)
                self.info_message(
                     "val_auc improved from {:.4f} to {:.4f}. Saved model to '{}'", 
                    self.best_valid_score, valid_auc, self.lastmodel
                )
                self.best_valid_score = valid_auc
                self.n_patience = 0
            else:
                self.n_patience += 1
            
            if self.n_patience >= patience:
                self.info_message("\nValid auc didn't improve last {} epochs.", patience)
                break
            
    def train_epoch(self, train_loader):
        self.model.train()
        t = time.time()
        sum_loss = 0

        for step, batch in enumerate(train_loader, 1):
            X = batch["X"].to(self.device)
            targets = batch["y"].to(self.device)
            self.optimizer.zero_grad()
            with autocast():
                outputs = self.model(X).squeeze(1)
                loss = self.criterion(outputs, targets)
                
            self.scaler.scale(loss).backward()

            sum_loss += loss.detach().item()

            self.scaler.step(self.optimizer)
            
            self.scaler.update()
            
            message = 'Train Step {}/{}, train_loss: {:.4f}'
            self.info_message(message, step, len(train_loader), sum_loss/step, end="\r")
            
        self.lr_scheduler.step()
        
        return sum_loss/len(train_loader), int(time.time() - t)
    
    def valid_epoch(self, valid_loader):
        self.model.eval()
        t = time.time()
        sum_loss = 0
        y_all = []
        outputs_all = []

        for step, batch in enumerate(valid_loader, 1):
            with torch.no_grad():
                targets = batch["y"].to(self.device)
                output = self.model(batch["X"].to(self.device)).squeeze(1)
                loss = self.criterion(output, targets)
                output = torch.sigmoid(output)
                sum_loss += loss.detach().item()

                y_all.extend(targets.tolist())
                outputs_all.extend(output.tolist())

            message = 'Valid Step {}/{}, valid_loss: {:.4f}'
            self.info_message(message, step, len(valid_loader), sum_loss/step, end="\r")
            
        auc = roc_auc_score(y_all, outputs_all)
        
        return sum_loss/len(valid_loader), auc, int(time.time() - t)
    
    def save_model(self, n_epoch, save_path, loss, auc):
        self.lastmodel = f"{save_path}-e{n_epoch}-loss{loss:.3f}-auc{auc:.3f}.pth"
        torch.save(
            {
                "model_state_dict": self.model.state_dict(),
                "optimizer_state_dict": self.optimizer.state_dict(),
                "best_valid_score": self.best_valid_score,
                "n_epoch": n_epoch,
            },
            self.lastmodel,
        )
        
    def display_plots(self):
        plt.figure(figsize=(10,5))
        plt.title("Training and Validation Loss")
        plt.plot(self.val_losses,label="val")
        plt.plot(self.train_losses,label="train")
        plt.xlabel("iterations")
        plt.ylabel("Loss")
        plt.legend()
        plt.show()
        plt.close()
        
        plt.figure(figsize=(10,5))
        plt.title("Validation AUC-ROC")
        plt.plot(self.val_auc,label="val")
        plt.xlabel("iterations")
        plt.ylabel("AUC")
        plt.legend()
        plt.show()
        plt.close()
    
    @staticmethod
    def info_message(message, *args, end="\n"):
        print(message.format(*args), end=end)

Loss function with DCA penalty for better model calibration

In [ ]:
# https://github.com/GB-TonyLiang/DCA
def cross_entropy_with_dca_loss(logits, labels, pos_weights=None, alpha=1., beta=BETA):        
    ce = F.binary_cross_entropy_with_logits(logits, labels, pos_weight=None)

    confidences = torch.sigmoid(logits)
    predictions = torch.round(confidences)
    accuracies = predictions.eq(labels)
    mean_conf = confidences.float().mean()
    acc = accuracies.float().sum() / len(accuracies)
    dca = torch.abs(mean_conf - acc)
    loss = alpha * ce + beta * dca
    
    return loss

## train models

Train models with augmented data where each modality is independently transformed with the use of Monai dictionary transformations

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(df_train.shape, df_valid.shape)
display(df_train.head())
display(df_valid.head())

train_transforms = Compose([
    RandAffined(mri_types, prob=0.5, rotate_range=np.pi, shear_range=0.05, translate_range=0.05, scale_range=0.05),
    RandGibbsNoised(mri_types),
    RandGaussianSmoothd(mri_types),
    ConcatItemsd(mri_types, "img")
])

valid_transforms = Compose([
    ConcatItemsd(mri_types, "img")
])

train_data_retriever = Dataset(
    df_train["BraTS21ID"].values, 
    df_train["MGMT_value"].values, 
    transforms=train_transforms
)

valid_data_retriever = Dataset(
    df_valid["BraTS21ID"].values, 
    df_valid["MGMT_value"].values,
    transforms=valid_transforms
)

train_loader = torch_data.DataLoader(
    train_data_retriever,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=2,
)

valid_loader = torch_data.DataLoader(
    valid_data_retriever, 
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=2,
)

model = build_model()
model.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

criterion = cross_entropy_with_dca_loss

trainer = Trainer(
    model, 
    device, 
    optimizer, 
    criterion
)

history = trainer.fit(
    N_EPOCHS, 
    train_loader, 
    valid_loader, 
    f"model2d", 
    N_EPOCHS,
)

trainer.display_plots()